In [ ]:

import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import os
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline
import random
import math

In [ ]:
import torchvision
import torchvision.transforms as transforms

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device available now:', device)

Device available now: cuda


In [ ]:
image_transforms = transforms.Compose(
         [transforms.ToTensor(),
         transforms.CenterCrop(28),
         transforms.Normalize((0.5), (0.5))]
)

In [ ]:
batchsize = 256
rowsize = 28
colsize = 28
inchannels = 3
kernelsize = 5
stride = 1
padding = 0
poolsize = 3
fc_size1 = 384
fc_size2 = 192
no_classes = 10
epochs = 30

In [ ]:

train_ioc = []
test_ioc = 0
train_nn = []
noise = 40
test_nn = 0

In [ ]:
train = torchvision.datasets.CIFAR10(root='./', train=True, download=True, transform=image_transforms)
test = torchvision.datasets.CIFAR10(root='./', train=False, transform=image_transforms)



  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to ./


In [ ]:
traindata = torch.utils.data.DataLoader(train,batch_size=batchsize, shuffle=True, num_workers=2,pin_memory=True)
testdata = torch.utils.data.DataLoader(test,batch_size=batchsize, shuffle=False, num_workers=2,pin_memory=True)

In [ ]:
# for i,data in enumerate(traindata):
#   #data[0] , data[1]
#   for j in range(len(data)):
#     data[1][j] = random.randint(0, 9)

In [ ]:
class IOC_CNN(nn.Module):
  def __init__(self,inchannels,kernelsize,stride,padding,poolsize,fc_size1,fc_size2,no_classes):
    super(IOC_CNN, self).__init__()
    self.conv1 = nn.Conv2d(inchannels,24,kernelsize,stride,padding)
    self.pool1 = nn.MaxPool2d(poolsize,2)
    self.BatchNorm1 = nn.BatchNorm2d(24)
    self.conv2 = nn.Conv2d(24,48,kernelsize,stride,padding)
    self.pool2 = nn.MaxPool2d(poolsize,2)
    self.BatchNorm2 = nn.BatchNorm2d(48)
    self.fc1 = nn.Linear(432,fc_size1)
    self.fc2 = nn.Linear(fc_size1,fc_size2)
    self.fc3 = nn.Linear(fc_size2,no_classes)


  def forward(self, x):
    x = F.elu(self.conv1(x))
    x = self.pool1(x)
    x = self.BatchNorm1(x)
    x = F.elu(self.conv2(x))
    x = self.pool2(x)
    x = self.BatchNorm2(x)
    x = torch.flatten(x,1)
    x = F.elu(self.fc1(x))
    x = F.elu(self.fc2(x))
    x = F.elu(self.fc3(x))
    return x

In [ ]:
class NN_CNN(nn.Module):
  def __init__(self,inchannels,kernelsize,stride,padding,poolsize,fc_size1,fc_size2,no_classes):
    super(NN_CNN, self).__init__()
    self.conv1 = nn.Conv2d(inchannels,24,kernelsize,stride,padding)
    self.pool1 = nn.MaxPool2d(poolsize,2)
    self.BatchNorm1 = nn.BatchNorm2d(24)
    self.conv2 = nn.Conv2d(24,48,kernelsize,stride,padding)
    self.pool2 = nn.MaxPool2d(poolsize,2)
    self.BatchNorm2 = nn.BatchNorm2d(48)
    self.fc1 = nn.Linear(432,fc_size1)
    self.fc2 = nn.Linear(fc_size1,fc_size2)
    self.fc3 = nn.Linear(fc_size2,no_classes)


  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool1(x)
    x = self.BatchNorm1(x)
    x = F.relu(self.conv2(x))
    x = self.pool2(x)
    x = self.BatchNorm2(x)
    x = torch.flatten(x,1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    return x

In [ ]:
ioc_model = IOC_CNN(inchannels,kernelsize,stride,padding,poolsize,fc_size1,fc_size2,no_classes).to(device)
nn_model = NN_CNN(inchannels,kernelsize,stride,padding,poolsize,fc_size1,fc_size2,no_classes).to(device)
# model = SimpleAlex(no_classes).to(device)

In [ ]:
ioc_optimizer = torch.optim.Adam(ioc_model.parameters(),lr= 0.0001)
nn_optimizer = torch.optim.Adam(nn_model.parameters(),lr= 0.0001)
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:

for i in range(epochs):
  Loss = 0
  num_correct=0
  tot = 0
  for i,data in enumerate(traindata):
    # iterating through every batch
    x_data = data[0].to(device)
    labels = data[1].to(device)
    ioc_optimizer.zero_grad()
    outputs = ioc_model(x_data)
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
    loss = criterion(outputs, labels)
    loss.backward()
    ioc_optimizer.step()
    Loss += loss
    for w in ioc_model.parameters():
      p = w
      torch.where(w > 0, w, torch.exp(p))
  train_ioc.append(num_correct/tot)    
  print("Accuracy for exponentiation on negative weights : ",num_correct/tot)
  print("Loss for exponentiation on negative weights : ",loss.item())
  

Accuracy for exponentiation on negative weights :  0.40956
Loss for exponentiation on negative weights :  1.4071670770645142
Accuracy for exponentiation on negative weights :  0.53332
Loss for exponentiation on negative weights :  1.3466993570327759
Accuracy for exponentiation on negative weights :  0.57734
Loss for exponentiation on negative weights :  1.0971719026565552
Accuracy for exponentiation on negative weights :  0.60452
Loss for exponentiation on negative weights :  1.1211843490600586
Accuracy for exponentiation on negative weights :  0.62606
Loss for exponentiation on negative weights :  0.9399492144584656
Accuracy for exponentiation on negative weights :  0.64152
Loss for exponentiation on negative weights :  0.9627361297607422
Accuracy for exponentiation on negative weights :  0.65424
Loss for exponentiation on negative weights :  0.9677961468696594
Accuracy for exponentiation on negative weights :  0.66712
Loss for exponentiation on negative weights :  1.122461199760437
A

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    confidence = 0
    for images, labels in testdata:
        images = images.to(device)
        labels = labels.to(device)
        outputs = ioc_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for i in range(len(predicted)):
          sum = torch.sum(outputs[i])
          confidence = confidence + (outputs[i][predicted[i]]/sum)
    test_ioc = correct/total
    confidence = confidence/total
    print("Test Accuracy for exponentiation of weights for IOC Architecture : ",100 * correct / total)
    print("The confidence Value for IOC Architecture : ",confidence.item())



Test Accuracy for exponentiation of weights for IOC Architecture :  68.94
The confidence Value for IOC Architecture :  0.5515899062156677


In [ ]:

for i in range(epochs):
  Loss = 0
  num_correct=0
  tot = 0
  for i,data in enumerate(traindata):
    # iterating through every batch
    x_data = data[0].to(device)
    labels = data[1].to(device)
    nn_optimizer.zero_grad()
    outputs = nn_model(x_data)
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
    loss = criterion(outputs, labels)
    loss.backward()
    nn_optimizer.step()
    Loss += loss
  train_nn.append(num_correct/tot)    
  print("Accuracy for exponentiation on negative weights : ",num_correct/tot)
  print("Loss for exponentiation on negative weights : ",loss.item())
  

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testdata:
        images = images.to(device)
        labels = labels.to(device)
        outputs = nn_model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for i in range(len(predicted)):
          sum = torch.sum(outputs[i])
          confidence = confidence + (outputs[i][predicted[i]]/sum)
    test_nn = correct/total
    confidence = confidence/total
    print("Test Accuracy for exponentiation of weights with NN Architecture : ",100 * correct / total)
    print("The confidence Value for IOC Architecture : ",confidence.item())


Test Accuracy for exponentiation of weights with NN Architecture :  66.01
The confidence Value for IOC Architecture :  0.4088934361934662
